# 爬取網站 : [IT邦幫忙](https://ithelp.ithome.com.tw/)
## 抓取內容 : 文章的標題、標籤、詢問日期、瀏覽次數
### P.S. 為了避免出現不必要的格式錯誤，將標題、標籤利用REGEX替換特定符號，替換前及替換後如下
### ( (換行符號(\n) -> EOF)　,　(Tab(\t) ->     (四個空白))　,　(, -> ，) )

In [3]:
import urllib.request as req #引入urllib模組中函式名稱為request的函式為req
import bs4 #引入bs4模組
import re
#定義函式getData，用以取得每一頁所有標題、標籤及內文。參數為網址
def getData(url,count,pages):

    #建立一個Request物件，附加Request Headers的資訊
    request = req.Request(url,headers = {
        "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36"
    })
    
    #打開request
    with req.urlopen(request) as response:
        data = response.read().decode("utf-8")

    root = bs4.BeautifulSoup(data,"html.parser") #讓BeautifulSoup解析HTML格式文件
    all_titles = root.find_all("h3",class_="qa-list__title") #以列表形式找出所有class_=title的h3標籤


    #開啟finalproject.txt並用a模式保留原本資料並將新的資料寫入
    with open("csvpy.csv",mode = "a",encoding = "big5") as ftext:
        #此迴圈會找出該頁所有標題、標籤及內文
        for title in all_titles:
            url2 = title.a["href"] #將內文的網址存到url2裡

            #建立一個Request物件，附加Request Headers的資訊
            request2 = req.Request(url2,headers = {
                "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36"
            })

            #打開request2
            with req.urlopen(request2) as response2:
                data2 = response2.read().decode("utf-8")
            
            root2 = bs4.BeautifulSoup(data2,"html.parser") #讓BeautifulSoup解析HTML格式文件
            tags = root2.find("div",class_="qa-header__tagGroup") #以列表形式找出所有 class_=qa-header__tagGroup 的 div 標籤
            time_and_view = root2.find("div",class_="qa-header__info").text.split() #以列表形式找出所有 class_=qa-header__info 的 div 標籤
            
            #利用regex將會影響CSV的符號做替換
            title.a.string = re.sub(r",","，",title.a.string)

            set_of_tags = set(tags.text.split("\n")) #利用集合將重複出現的標籤強制讓它出現一次
            #將集合內的資料重新裝回list內
            list_of_tags = []
            for i in set_of_tags:
                list_of_tags.append(i)
            
            #將標籤、詢問日期、瀏覽次數及內文寫進CSV檔
            for tag in list_of_tags[1:]:
                tag = re.sub(r",","，",tag)
                ftext.write(title.a.string+",")
                ftext.write(tag + ",")
                ftext.write(time_and_view[1]+",")
                ftext.write(time_and_view[-2]+",")
                ftext.write(contents+"\n")
                print("標題 : "+title.a.string)
                print("標籤 : "+tag)
                print("詢問日期 : "+time_and_view[1])
                print("瀏覽次數 : "+time_and_view[-2])
                print("\n\n")


    #抓取下一頁的連結
    nextLink = root.find("a",string="下一頁") #找到內文是 下一頁 的標籤a
    return nextLink["href"] #回傳下一頁的網址



# 主程式
### 要求使用者輸入想抓取多少頁數的資料

In [4]:
#將每列代表的意義寫入CSV檔
with open("csvpy.csv",mode = "a",encoding = "big5") as ftext:
    ftext.write("標題,標籤,詢問時間,瀏覽次數,內文\n")

pageURL = "https://ithelp.ithome.com.tw/" #將起始網頁的網址存到pageURL裡
count = 0 #count用來表示想抓取幾頁
pages = int(input("Please enter how many pages you want to crawl:")) #輸入要抓取幾頁資料

#重複抓取每頁資料
while count<pages:
    pageURL = getData(pageURL,count,pages) #將 return 回來的網址覆蓋到pageURL上，以便重複利用
    count += 1 #執行完一次後增加1，以確保能抓到預期的頁數

Please enter how many pages you want to crawl:1
標題 : 爬蟲問題
標籤 : big5
詢問日期 : 2022-11-02
瀏覽次數 : 25
內文 : 
我原本想用爬蟲把IT邦邦忙網站中的文章"標題"，"標籤"，"詢問時間"，"瀏覽次數"，還有"問題敘述"抓下來用csv檔儲存，禮拜一用的時候好好的，結果今天用就突然掛掉。
我看了一下error好像是有人文章裡的"問題敘述"格式不符合csv檔用的文字編碼big5
跑出的error長這樣 :
UnicodeEncodeError: 'big5' codec can't encode character '\u6ca1' in position 3244: illegal multibyte sequence
想請問這段error是什麼意思?要如何解決?
更:我自己這篇文章的東西有抓下來，到下面那篇文章的"問題敘述"就抓不下來了，應該是那篇文章的內容有影響到
contents = root2.find("div",class_="markdown__style").text #找出內文

這段是我用來抓"問題敘述"的程式





標題 : 爬蟲問題
標籤 : csv
詢問日期 : 2022-11-02
瀏覽次數 : 25
內文 : 
我原本想用爬蟲把IT邦邦忙網站中的文章"標題"，"標籤"，"詢問時間"，"瀏覽次數"，還有"問題敘述"抓下來用csv檔儲存，禮拜一用的時候好好的，結果今天用就突然掛掉。
我看了一下error好像是有人文章裡的"問題敘述"格式不符合csv檔用的文字編碼big5
跑出的error長這樣 :
UnicodeEncodeError: 'big5' codec can't encode character '\u6ca1' in position 3244: illegal multibyte sequence
想請問這段error是什麼意思?要如何解決?
更:我自己這篇文章的東西有抓下來，到下面那篇文章的"問題敘述"就抓不下來了，應該是那篇文章的內容有影響到
contents = root2.find("div",class_="markdown__style").text #找出內文

這段是我用來抓"問題敘述"的程式





標題 : 爬蟲問題

UnicodeEncodeError: 'big5' codec can't encode character '\u6ca1' in position 3156: illegal multibyte sequence